<a href="https://colab.research.google.com/github/99ashmeet/99ashmeet.github.io/blob/master/DeepFake_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
!mkdir deepFake_detection_algo

In [2]:
!cp /content/drive/MyDrive/minor_project/deepfake_database.zip /content/deepFake_detection_algo

In [3]:
cd deepFake_detection_algo/

/content/deepFake_detection_algo


In [4]:
!7z x deepfake_database.zip


7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.20GHz (406F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 192901191 bytes (184 MiB)

Extracting archive: deepfake_database.zip
 21% 4096 Open              --
Path = deepfake_database.zip
Type = zip
Physical Size = 192901191

  0%      5% 606 - deepfake_database/deepfake_database/train:test/df/df00601.jpg                                                                          8% 1017 - deepfake_database/deepfake_database/train:test/df/df01012.jpg                                                                         

In [5]:
!cp /content/deepFake_detection_algo/deepfake_database/deepfake_database/validation/fake/* /content/deepFake_detection_algo/deepfake_database/deepfake_database/train:test/fake
!cp /content/deepFake_detection_algo/deepfake_database/deepfake_database/validation/real/* /content/deepFake_detection_algo/deepfake_database/deepfake_database/train:test/real

In [6]:
cd /content/deepFake_detection_algo

/content/deepFake_detection_algo


In [7]:
!mkdir deepfakedetection

In [8]:
!mv /content/deepFake_detection_algo/deepfake_database/deepfake_database/train:test/* /content/deepFake_detection_algo/deepfakedetection 

In [9]:
!mkdir images
!mkdir splits

In [10]:
import cv2, csv, os, shutil
import numpy as np
from PIL import Image
path = os.getcwd()

dataset_deepfakedetection = path + '/deepfakedetection'
datasets = [dataset_deepfakedetection]
output_folder = path + '/images/'
with open(output_folder + 'images.csv', 'a') as csv_file:
  filewriter = csv.writer(csv_file, delimiter=',')
  filewriter.writerow(['image_id','fake','original_dataset'])
for dataset in datasets:
  for real_image in os.listdir(dataset + '/real'):
    if real_image.endswith('.jpg'):

      shutil.move(dataset + '/real/' + real_image, output_folder + real_image)
      with open(output_folder + 'images.csv', 'a') as csv_file:
        filewriter = csv.writer(csv_file, delimiter=',')
        filewriter.writerow([real_image, 0, dataset.rsplit('/', 1)[1]])
    else:
      continue
  for fake_image in os.listdir(dataset + '/fake'):
    if fake_image.endswith('.jpg'):
     
      shutil.move(dataset + '/fake/' + fake_image, output_folder + fake_image)
      with open(output_folder + 'images.csv', 'a') as csv_file:
        filewriter = csv.writer(csv_file, delimiter=',')
        filewriter.writerow([fake_image, 1, dataset.rsplit('/', 1)[1]])
    else:
      continue

print("Files have been successfully moved into the output folder and all information put into a CSV file.")

Files have been successfully moved into the output folder and all information put into a CSV file.


In [11]:
import pandas as pd 
df=pd.read_csv('images/images.csv')
df = df.sample(frac=1)

In [35]:
cd /content/deepFake_detection_algo/images

/content/deepFake_detection_algo/images


In [12]:
df_1 = df.iloc[:15564,:] 
df_2 = df.iloc[15564:17510,:] 
df_3 = df.iloc[17510:,:]

df_1.to_csv('../train_split.csv', index=False) 
df_2.to_csv('../val_split.csv', index=False) 
df_3.to_csv('../test_split.csv', index=False) 

In [13]:
!cp /content/deepFake_detection_algo/splits/* /content/drive/MyDrive/minor_project/splits

In [5]:
cd /content/deepFake_detection_algo

/content/deepFake_detection_algo


In [6]:
import pandas as  pd
import numpy as np
import cv2

def makeDataSet(file_csv):

  x = []
  y = []

  for i in file_csv.index:

    if(file_csv['image_id'][i]=="image_id"):
      continue;
  
    img_address = "images/" + str(file_csv['image_id'][i]);

    img = cv2.imread(img_address)
    img = cv2.resize(img,(256,256))

    img_array = np.array(img)

    x.append(img_array)
    y.append(file_csv['fake'][i])  
  
  x = np.array(x)
  y = np.array(y)

  return x,y

In [7]:
x_train,y_train = makeDataSet(pd.read_csv('/content/deepFake_detection_algo/splits/train_split.csv'))
x_validate,y_validate = makeDataSet(pd.read_csv('/content/deepFake_detection_algo/splits/val_split.csv'))
x_test,y_test = makeDataSet(pd.read_csv('/content/deepFake_detection_algo/splits/test_split.csv'))

In [9]:
cd /content/deepFake_detection_algo

/content/deepFake_detection_algo


In [10]:
print(x_train.shape)
print(y_train.shape)
print(x_validate.shape)
print(y_validate.shape)
print(x_test.shape)
print(y_test.shape)

(15564, 256, 256, 3)
(15564,)
(1946, 256, 256, 3)
(1946,)
(1946, 256, 256, 3)
(1946,)


In [32]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import MaxPool2D
from keras.layers import Dropout
from keras.layers import Flatten
from keras import backend as K
import tensorflow as tf
import numpy as np

In [33]:
K.clear_session()
tf.compat.v1.GPUOptions(allow_growth=True)

allow_growth: true

In [34]:
model= Sequential()

In [35]:
cnn1= Conv2D(filters=16, kernel_size=(5,5), strides=(1,1), padding= "SAME",activation="relu", use_bias=True, input_shape=(256,256,3))
model.add(cnn1)
model.add(MaxPool2D(pool_size=(2,2), strides=(1,1)))
model.add(Conv2D(filters=32, kernel_size=(5,5), strides=(1,1), padding= "SAME",activation="relu", use_bias=True))
model.add(MaxPool2D(pool_size=(2,2), strides=(1,1)))
model.add(Conv2D(filters=64, kernel_size=(5,5), strides=(1,1), padding= "SAME",activation="relu", use_bias=True))
model.add(MaxPool2D(pool_size=(2,2), strides=(1,1)))
model.add(Conv2D(filters=128, kernel_size=(5,5), strides=(1,1), padding= "SAME",activation="relu", use_bias=True))
model.add(MaxPool2D(pool_size=(2,2), strides=(1,1)))
model.add(Flatten())
model.add(Dense(units=1,activation="sigmoid"))

In [36]:
model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'], experimental_steps_per_execution = 2)

In [37]:
model.fit(x_train,y_train, epochs=10, validation_data=(x_validate,y_validate))

Epoch 1/10
487/487 [==============================] - 265s 544ms/step - loss: 150.6881 - accuracy: 0.6529 - val_loss: 0.5772 - val_accuracy: 0.7143
Epoch 2/10
487/487 [==============================] - 265s 544ms/step - loss: 0.4537 - accuracy: 0.7906 - val_loss: 0.3271 - val_accuracy: 0.8643
Epoch 3/10
487/487 [==============================] - 265s 544ms/step - loss: 0.2907 - accuracy: 0.8791 - val_loss: 0.2891 - val_accuracy: 0.8823
Epoch 4/10
487/487 [==============================] - 265s 545ms/step - loss: 0.2388 - accuracy: 0.9027 - val_loss: 0.2431 - val_accuracy: 0.8947
Epoch 5/10
487/487 [==============================] - 265s 544ms/step - loss: 0.2149 - accuracy: 0.9124 - val_loss: 0.2870 - val_accuracy: 0.8905
Epoch 6/10
487/487 [==============================] - 265s 545ms/step - loss: 0.1808 - accuracy: 0.9257 - val_loss: 0.2872 - val_accuracy: 0.8988
Epoch 7/10
487/487 [==============================] - 265s 545ms/step - loss: 0.1830 - accuracy: 0.9322 - val_loss: 0.2998

In [ ]:
# model = keras.models.load_model("DeepFake_Detection")

In [38]:
evaluated= model.evaluate(x_test,y_test)

61/61 [==============================] - 9s 148ms/step - loss: 0.3478 - accuracy: 0.8941


In [39]:
print('Test loss:', evaluated[0])
print('Test accuracy:', evaluated[1]*100,"%")

Test loss: 0.3478047549724579
Test accuracy: 89.41418528556824 %


In [40]:
model.save("DeepFake_Detection")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: DeepFake_Detection/assets


In [41]:
!cp -r /content/deepFake_detection_algo/DeepFake_Detection /content/drive/MyDrive/minor_project

In [42]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 256, 256, 16)      1216      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 255, 255, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 255, 255, 32)      12832     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 254, 254, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 254, 254, 64)      51264     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 253, 253, 64)      0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 253, 253, 128)     2